### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

agents talk to each other and judge agent orchestrates the whole exchange

Player1Agent -> Pro side debater
Player2Agent -> Con side debater
Judge -> evaluator who gives final decisions

3 agents, all subclassses of RoutedAgent (special AutoGen agent that can route messages between agents in a distributed system)

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some key pros of using AutoGen in your AI Agent project:

1. **Improved Efficiency**: AutoGen utilizes asynchronous messaging and event-driven architecture, which allows for faster and more efficient communication between agents. This results in quicker response times and overall better performance.

2. **Scalability**: The framework is designed to be modular and extensible, making it easier to create scalable systems. This means you can grow your project and add new features or agents without significant rework.

3. **Customization**: AutoGen allows for a high degree of customization, enabling developers to tailor agents to specific use cases and requirements, which can enhance the overall effectiveness of the AI systems.

4. **Interoperability**: It supports various protocols and integrates well with different platforms, which can help in developing agents that interact seamlessly with diverse systems.

5. **Simplified Development**: The framework simplifies the development process, reducing the time and effort required to build and deploy AI agents, thus accelerating project timelines.

These advantages can significantly contribute to the success of your AI Agent project by improving both development efficiency and operational effectiveness. 

TERMINATE

## Cons of AutoGen:
Here are several reasons against choosing AutoGen for an AI Agent project:

1. **Technical Expertise Required**: AutoGen typically demands a solid understanding of programming and AI concepts. This might pose a barrier for teams without sufficient technical know-how.

2. **Complexity**: The extensive customization options and flexibility can lead to increased complexity in implementation. This could overwhelm new users or those unfamiliar with similar frameworks.

3. **Potential for Overhead**: Customizing and optimizing the framework for specific use cases may require substantial engineering effort, which could detract from focusing on the core project goals.

4. **Community and Support**: Depending on its adoption rate, there may be limited community support or documentation available compared to more established frameworks, making troubleshooting and learning more difficult.

5. **Performance Issues**: If not configured correctly, users could face potential performance bottlenecks or inefficiencies, leading to a less responsive AI agent.

6. **Integration Challenges**: Integrating AutoGen with other existing systems or platforms may introduce additional challenges, requiring extra time and resources.

Considering these cons will assist in making an informed decision regarding the use of AutoGen for your project. 

TERMINATE



## Decision:

Based on the research provided by the team, I recommend using AutoGen for the project. 

**Rationale**: The benefits of improved efficiency, scalability, customization, and simplified development present a compelling case for adopting AutoGen. These advantages can significantly enhance our project's operational effectiveness and speed up timelines. While the complexities and technical expertise required are valid concerns, they can be mitigated with proper training and support. Ultimately, the pros outweigh the cons, making AutoGen a strong candidate for our AI Agent project.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()